<a href="https://colab.research.google.com/github/jaideep11061982/GenAINotebooks/blob/main/Google_Gemini_long_context_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Long-context experiments with Google Gemini

Google's Gemini Pro 1.5 has an impressive 1,000,000 token context window. But when is such a long context window useful? In this quick demonstration, we show how combining existing RAG pipelines with a long context window can produce superior results.

First we install our dependencies.

In [ ]:
!pip install llama-index-core
!pip install llama-index-llms-gemini
!pip install llama-index-embeddings-huggingface
!pip install llama_index.readers.file

In [ ]:
from llama_index.core import Settings, SimpleDirectoryReader, VectorStoreIndex
from llama_index.core.callbacks import CallbackManager, TokenCountingHandler
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.agent import ReActAgent
from llama_index.llms.gemini import Gemini
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from google.colab import userdata
import os

# Import data
The data we're using is a series of complex PDFs we found detailing budgets for San Francisco from 2016 to 2023. It's a tough task -- each PDF is formatted differently, each refers to the years before and after for comparison, and there's a missing year where the budget numbers have to be inferred. The combined PDFs are also enormous -- even with a 1m token window, you can't just send the full text of all the documents all at once. Perfect for this test!

In [ ]:
!mkdir data
!wget "https://www.dropbox.com/scl/fi/xt3squt47djba0j7emmjb/2016-CSF_Budget_Book_2016_FINAL_WEB_with-cover-page.pdf?rlkey=xs064cjs8cb4wma6t5pw2u2bl&dl=0" -O data/budget_2016.pdf
!wget "https://www.dropbox.com/scl/fi/jvw59g5nscu1m7f96tjre/2017-Proposed-Budget-FY2017-18-FY2018-19_1.pdf?rlkey=v988oigs2whtcy87ti9wti6od&dl=0" -O data/budget_2017.pdf
!wget "https://www.dropbox.com/scl/fi/izknlwmbs7ia0lbn7zzyx/2018-o0181-18.pdf?rlkey=p5nv2ehtp7272ege3m9diqhei&dl=0" -O data/budget_2018.pdf
!wget "https://www.dropbox.com/scl/fi/1rstqm9rh5u5fr0tcjnxj/2019-Proposed-Budget-FY2019-20-FY2020-21.pdf?rlkey=3s2ivfx7z9bev1r840dlpbcgg&dl=0" -O data/budget_2019.pdf
!wget "https://www.dropbox.com/scl/fi/7teuwxrjdyvgw0n8jjvk0/2021-AAO-FY20-21-FY21-22-09-11-2020-FINAL.pdf?rlkey=6br3wzxwj5fv1f1l8e69nbmhk&dl=0" -O data/budget_2021.pdf
!wget "https://www.dropbox.com/scl/fi/zhgqch4n6xbv9skgcknij/2022-AAO-FY2021-22-FY2022-23-FINAL-20210730.pdf?rlkey=h78t65dfaz3mqbpbhl1u9e309&dl=0" -O data/budget_2022.pdf
!wget "https://www.dropbox.com/scl/fi/vip161t63s56vd94neqlt/2023-CSF_Proposed_Budget_Book_June_2023_Master_Web.pdf?rlkey=hemoce3w1jsuf6s2bz87g549i&dl=0" -O data/budget_2023.pdf

# Token counting

To demonstrate exactly how long the context window we're using is going to be, we've turned on token counting for this experiment.

In [ ]:
token_counter = TokenCountingHandler(
    verbose=True
)

Settings.callback_manager = CallbackManager([token_counter])


# Embed documents

We're using local embeddings.

In [ ]:
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# Set chunk size and overlap

To maximize accuracy, we're using a relatively small chunk size. Now we ingest the documents and let the embedder do its work.

# Initialize Gemini

Let's bring in Gemini Pro 1.5. You can [get your own API key](https://aistudio.google.com/app/apikey) now that it's generally available!

In [ ]:
Settings.chunk_size = 512
Settings.chunk_overlap = 50

documents = SimpleDirectoryReader("./data").load_data()
index = VectorStoreIndex.from_documents(documents)

Embedding Token Usage: 2055
Embedding Token Usage: 3476
Embedding Token Usage: 3977
Embedding Token Usage: 3718
Embedding Token Usage: 3110
Embedding Token Usage: 2731
Embedding Token Usage: 3685
Embedding Token Usage: 3933
Embedding Token Usage: 3267
Embedding Token Usage: 4099
Embedding Token Usage: 4375
Embedding Token Usage: 4925
Embedding Token Usage: 4483
Embedding Token Usage: 4812
Embedding Token Usage: 4673
Embedding Token Usage: 4930
Embedding Token Usage: 4270
Embedding Token Usage: 4628
Embedding Token Usage: 4622
Embedding Token Usage: 4702
Embedding Token Usage: 3517
Embedding Token Usage: 2789
Embedding Token Usage: 3446
Embedding Token Usage: 3284
Embedding Token Usage: 3302
Embedding Token Usage: 3634
Embedding Token Usage: 3185
Embedding Token Usage: 2950
Embedding Token Usage: 3592
Embedding Token Usage: 3251
Embedding Token Usage: 3740
Embedding Token Usage: 3584
Embedding Token Usage: 2196
Embedding Token Usage: 3140
Embedding Token Usage: 3053
Embedding Token Usag

In [ ]:
os.environ["GOOGLE_API_KEY"] = userdata.get('google-api-key')
Settings.llm = Gemini(
    model_name="models/gemini-1.5-pro-latest",
    temperature=0.2
)

# Create an agent

We generally get better results from RAG if we use an agentic approach, since it can reflect and ask multiple questions until it gets the answer rather than trying to get the answer in one shot. Here we're creating a query engine from our ingested data and setting `similarity_top_k` to 10, meaning the RAG engine will retrieve the 10 most relevant pieces of embedded context and supply them to Gemini to attempt to answer the question.

In [ ]:
query_engine_10 = index.as_query_engine(similarity_top_k=10)
query_engine_tools = [
    QueryEngineTool(
        query_engine=query_engine_10,
        metadata=ToolMetadata(
            name="sf_budgets",
            description=(
                "Has information about the budget of San Francisco, with documents for every year from 2016 to 2023."
            ),
        ),
    ),
]

agent_10 = ReActAgent.from_tools(
    query_engine_tools,
    verbose=True,
    max_iterations=100
)

response = agent_10.chat("What was the budget of San Francicisco for each fiscal year from 2016 to 2023?")
print(str(response))


LLM Prompt Token Usage: 478
LLM Completion Token Usage: 63
Thought: The current language of the user is: english. I need to use a tool to find the budget information for San Francisco.
Action: sf_budgets
Action Input: {'input': 'What was the budget of San Francisco for each fiscal year from 2016 to 2023?'}
Embedding Token Usage: 20
LLM Prompt Token Usage: 4793
LLM Completion Token Usage: 53
Observation: This question cannot be answered from the given source. This document only provides information on the budgets for fiscal years 2019-2020, 2020-2021, 2023-2024, and 2024-2025. 

LLM Prompt Token Usage: 601
LLM Completion Token Usage: 92
Thought: I cannot answer the question with the provided tools.
Answer: I can't answer your question. While I have information on the San Francisco budget, I only have documents for fiscal years 2019-2020, 2020-2021, 2023-2024, and 2024-2025. I do not have information for fiscal years 2016-2019 or 2021-2023.
I can't answer your question. While I have info

# Bad results at 10

You can see that the engine delivered 4,793 tokens worth of context to Gemini to try and answer the question but the LLM decided it didn't have enough information to answer the question -- specifically, it mentions that it is missing whole years.

# Try again at 100

So we're going to create a second agent, identical to the first but set to retrieve 100 pieces of context.

In [ ]:
query_engine_100 = index.as_query_engine(similarity_top_k=100)
query_engine_tools = [
    QueryEngineTool(
        query_engine=query_engine_100,
        metadata=ToolMetadata(
            name="sf_budgets",
            description=(
                "Has information about the budget of San Francisco, with documents for every year from 2016 to 2023."
            ),
        ),
    ),
]

agent_100 = ReActAgent.from_tools(
    query_engine_tools,
    verbose=True,
    max_iterations=100
)

response = agent_100.chat("What was the budget of San Francicisco for each fiscal year from 2016 to 2023?")
print(str(response))

LLM Prompt Token Usage: 478
LLM Completion Token Usage: 63
Thought: The current language of the user is: english. I need to use a tool to find the budget information for San Francisco.
Action: sf_budgets
Action Input: {'input': 'What was the budget of San Francisco for each fiscal year from 2016 to 2023?'}
Embedding Token Usage: 20
LLM Prompt Token Usage: 45089
LLM Completion Token Usage: 35
Observation: The provided text spans multiple San Francisco budget proposals, but does not contain the final adopted budget amounts for each fiscal year from 2016 to 2023. 

LLM Prompt Token Usage: 583
LLM Completion Token Usage: 56
Thought: I cannot answer the question with the provided tools.
Answer: I apologize, but I cannot provide the exact budget amounts for San Francisco for each fiscal year from 2016 to 2023. The available tool does not contain the final adopted budget numbers.
I apologize, but I cannot provide the exact budget amounts for San Francisco for each fiscal year from 2016 to 202

# Failure at 100

You can see this time `LLM Prompt Token Usage` was 45,089 -- roughly 10 times as much -- but the LLM still doesn't have quite enough data to answer the question. It's no longer complaining about missing years though, so the greater context has improved matters.

# Try again at 1000

So let's go again, this time with a truly generous 1000 pieces of context!

In [ ]:
query_engine_1000 = index.as_query_engine(similarity_top_k=1000)
query_engine_tools = [
    QueryEngineTool(
        query_engine=query_engine_1000,
        metadata=ToolMetadata(
            name="sf_budgets",
            description=(
                "Has information about the budget of San Francisco, with documents for every year from 2016 to 2023."
            ),
        ),
    ),
]

agent_1000 = ReActAgent.from_tools(
    query_engine_tools,
    verbose=True,
    max_iterations=100
)

response = agent_1000.chat("What was the budget of San Francicisco for each fiscal year from 2016 to 2023? Fetch each year separately.")
print(str(response))


LLM Prompt Token Usage: 483
LLM Completion Token Usage: 63
Thought: The current language of the user is: english. I need to use a tool to find the budget information for San Francisco.
Action: sf_budgets
Action Input: {'input': 'What was the budget of San Francisco for each fiscal year from 2016 to 2023?'}
Embedding Token Usage: 20
LLM Prompt Token Usage: 436369
LLM Completion Token Usage: 56
Observation: The provided text does not contain the budget of San Francisco for each fiscal year from 2016 to 2023. However, it does mention that the budget for fiscal years 2023-24 and 2024-25 is $14.6 billion. 

LLM Prompt Token Usage: 609
LLM Completion Token Usage: 66
Thought: The tool could not find the exact budget numbers for each year. I will try a different approach to get the information.
Action: sf_budgets
Action Input: {'input': 'Provide a summary of the budget for each fiscal year from 2016 to 2023 in San Francisco.'}
Embedding Token Usage: 22
LLM Prompt Token Usage: 427158
LLM Comple

# Success!

This time our retriever delivered 436,396 tokens of context -- nearly half the 1m window -- and now the agent works! It's able to provide us with budget number for every year. Also note that the agentic strategy helped us here -- it didn't get the answer the first time it tried the question, so it tried phrasing the question a different way and got the answer on the second try.